In [1]:
import json
import pandas as pd
import pathlib as pl

In [2]:
Project_Area = 'Sacramento'
root_dir = pl.Path(r'C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\{0}'.format(Project_Area))
outputs_dir = root_dir/'Outputs'

In [3]:
forcing_files = []
for f in outputs_dir.glob('**/*.json'):
    if 'Forcing' in f.parent.stem:
        forcing_files.append(f)
        print(f)

C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D23.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D24.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D25.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D26.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D27.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D28.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D29.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D30.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Outputs\Sacramento_P

In [4]:
models = []
domains = []
events = []

for forcing_file in forcing_files:
    _, Pluvial_Model, Domain =  forcing_file.stem.split('_')
    
    with open(forcing_file) as f:
        f_dic = json.load(f)
        
    durs = list(f_dic.keys())
    
    bcns = list(f_dic[durs[0]]['BCName'].keys())
    lbcns = [x for x in bcns if 'L' in x]
              
    if len(lbcns)!=0:
        models.append(Pluvial_Model)
        domains.append(Domain)
        select_lbcn = lbcns[0]
        abs_max = 0.0
        for d in durs:
            for k, v in f_dic[d]['BCName'][Domain].items():
                if sum(v)==0:
                    qmax = max(f_dic[d]['BCName'][select_lbcn][k])
                    if abs_max <= qmax:
                        abs_max = qmax
                        select_event = k
        events.append(select_event)            
        print('For {0} {1} the event with no excess rainfall which has the largest lateral inflow peak is {2} with {3} cfs'.format(Pluvial_Model, Domain, select_event, int(abs_max)))

For P01 D26 the event with no excess rainfall which has the largest lateral inflow peak is E1236 with 1832 cfs
For P01 D29 the event with no excess rainfall which has the largest lateral inflow peak is E1237 with 478 cfs
For P01 D30 the event with no excess rainfall which has the largest lateral inflow peak is E0098 with 976 cfs
For P02 D10 the event with no excess rainfall which has the largest lateral inflow peak is E0140 with 4451 cfs
For P02 D12 the event with no excess rainfall which has the largest lateral inflow peak is E0140 with 3209 cfs
For P02 D15 the event with no excess rainfall which has the largest lateral inflow peak is E0232 with 4543 cfs
For P02 D19 the event with no excess rainfall which has the largest lateral inflow peak is E0219 with 746 cfs
For P02 D22 the event with no excess rainfall which has the largest lateral inflow peak is E0219 with 6653 cfs
For P03 D37 the event with no excess rainfall which has the largest lateral inflow peak is E0197 with 2187 cfs
For 

In [5]:
df = pd.DataFrame()
df['Model'] = models
df['Domain'] = domains
df ['Event'] = events
df = df.set_index('Model')
df.to_csv(outputs_dir/'{0}_Events_LargestLat.csv'.format(Project_Area))
df.head()

,Domain,Event
Model,,
P01,D26,E1236
P01,D29,E1237
P01,D30,E0098
P02,D10,E0140
P02,D12,E0140


---
## End